<p style="font-weight:bold;"> <span style="font-size: 36px"> Scenario Parameters Import Test </span> </p>

In [0]:
#!eval-notebook "../Initialization/InitSystemorphBaseToMemory"

In [0]:
DataSource.Reset(x => x.ResetType<RawVariable>().ResetType<IfrsVariable>().ResetType<DataNodeState>());

# Setting Constants

In [0]:
var EnableScenarioBackup = Scenarios.EnableScenario;

In [0]:
Scenarios.EnableScenario = true;

In [0]:
var argsBestEstimate = new ImportArgs ("CH", 2020, 12, Periodicity.Quarterly, null, ImportFormats.Cashflow);
var argsScenarioActive = new ImportArgs ("CH", 2020, 12, Periodicity.Quarterly, "DNSCHAct", ImportFormats.Cashflow);
var argsScenarioInactive = new ImportArgs ("CH", 2020, 12, Periodicity.Quarterly, "DNSCHInact", ImportFormats.Cashflow);

# Best Estimate initialization

In [0]:
var dataNodeStateBestEstimateInit = @"
@@Main,,
ReportingNode,Year,Month
CH,2020,12
@@DataNodeState,,
DataNode,State,
DT1.1,Active,
DTR1.1,Active";

In [0]:
await Import.FromString(dataNodeStateBestEstimateInit).WithFormat(ImportFormats.DataNodeState).WithTarget(DataSource).ExecuteAsync()

In [0]:
var cashflowBestEstimateInit = @"
@@Main
ReportingNode,Year,Month,Scenario
CH,2020,12,
@@Cashflow
DataNode,AmountType,EstimateType,AocType,Novelty,AccidentYear,Values0,Values1,Values2,Values3,Values4,Values5,Values6,Values7,Values8,Values9,Values10,Values11,Values12,Values13,Values14,Values15,Values16,Values17,Values18,Values19,Values20,Values21,Values22,Values23
DT1.1,PR,BE,CL,C,,100,0,0,100,0,0,100,0,0,100,0,0,0,100,0,0,100,0,0,100,0,0,100,0
DT1.1,NIC,BE,CL,C,,-25,-25,-25,-25,-25,-25,-25,-25,-25,-25,-25,-25,-25,-25,-25,-25,-25,-25,-25,-25,-25,-25,-25,-25
DT1.1,CU,P,CL,C,,-10,-10,-10,-10,-10,-10,-10,-10,-10,-10,-10,-10,-10,-5,-5,-5,-5,-5,-5,-5,-5,-5,-5,-3
DT1.1,,RA,CL,C,,-2.5,-2.5,-2.5,-2.5,-2.5,-2.5,-2.5,-2.5,-2.5,-2.5,-2.5,-2.5,-2.5,-2.5,-2.5,-2.5,-2.5,-2.5,-2.5,-2.5,-2.5,-2.5,-2.5,-2.5
DTR1.1,PR,BE,CL,C,,50,0,0,50,0,0,50,0,0,50,0,0,50,0,0,50,0,0,50,0,0,50,0,0
DTR1.1,NIC,BE,CL,C,,12.5,12.5,12.5,12.5,12.5,12.5,12.5,12.5,12.5,12.5,12.5,12.5,12.5,12.5,12.5,12.5,12.5,12.5,12.5,12.5,12.5,12.5,12.5,12.5
DTR1.1,CU,P,CL,C,,-5,-5,-5,-5,-5,-5,-5,-5,-5,-5,-5,-5,-5,-5,-5,-5,-5,-5,-5,-5,-5,-5,-5,-5
DTR1.1,,RA,CL,C,,1.25,1.25,1.25,1.25,1.25,1.25,1.25,1.25,1.25,1.25,1.25,1.25,1.25,1.25,1.25,1.25,1.25,1.25,1.25,1.25,1.25,1.25,1.25,1.25";

In [0]:
await Import.FromString(cashflowBestEstimateInit).WithFormat(ImportFormats.Cashflow).WithTarget(DataSource).ExecuteAsync()

In [0]:
var actualsBestEstimateInit = @"
@@Main
ReportingNode,Year,Month,Scenario
CH,2020,12,
@@Actual
DataNode,AocType,AmountType,EstimateType,AccidentYear,Value
DT1.1,CF,ACA,A,,-10
DT1.1,CF,AEA,A,,-5
DT1.1,CF,NIC,A,,-280
DT1.1,CF,PR,A,,400
DTR1.1,CF,PR,A,,-200
DTR1.1,CF,NIC,A,,140";

In [0]:
await Import.FromString(actualsBestEstimateInit).WithFormat(ImportFormats.Actual).WithTarget(DataSource).ExecuteAsync()

In [0]:
await DataSource.Partition.SetAsync<PartitionByReportingNodeAndPeriod>(argsBestEstimate);
var ivsBestEstimateInit = await DataSource.Query<IfrsVariable>().ToArrayAsync();
var dnsBestEstimateInit = await DataSource.Query<DataNodeState>().ToArrayAsync();

In [0]:
dnsBestEstimateInit

# Scenario Data Node State import

In [0]:
var dataNodeStateActiveScenario = @"
@@Main
ReportingNode,Year,Month,Scenario,
CH,2020,12,DNSCHAct,
@@DataNodeState
DataNode,State,
DT1.2,Active,
DTR1.2,Active";

In [0]:
static ImportArgs myargs ;
static Dictionary<string, DataNodeData> mydict;
static HashSet<string> mytargetDataNodes;
static ImportArgs[] myallArgs;
static DataNodeState[] myCommittedStates;
static DataNodeState[] myToCommitStates;

In [0]:
public async static Task ValidateDataNodeStatesAsync(this IWorkspace workspace, Dictionary<string, DataNodeData> persistentDataNodeByDataNode)
{
    foreach(var importedDataNodeState in await workspace.Query<DataNodeState>().ToArrayAsync())
    {
        if(persistentDataNodeByDataNode.TryGetValue(importedDataNodeState.DataNode, out var currentPersistentDataNode))
        {
            if(importedDataNodeState.State < currentPersistentDataNode.State)
                ApplicationMessage.Log(Error.ChangeDataNodeState, importedDataNodeState.DataNode, 
                                                                  currentPersistentDataNode.State.ToString(), 
                                                                  importedDataNodeState.State.ToString());

            if(importedDataNodeState.State == currentPersistentDataNode.State)
                await workspace.DeleteAsync<DataNodeState>(importedDataNodeState);
        }
    }
}

In [0]:
Import.DefineFormat(ImportFormats.DataNodeState, async (options, dataSet) => {
    Activity.Start();
    var primaryArgs = GetArgsFromMain(dataSet) with {ImportFormat = ImportFormats.DataNodeState};
    myargs = primaryArgs;
    primaryArgs.ValidateArgsForPeriodAsync(options.TargetDataSource);
    if(ApplicationMessage.HasErrors()) return Activity.Finish();

    var workspace = Workspace.CreateNew();
    workspace.Initialize(x => x.FromSource(options.TargetDataSource)
                               .DisableInitialization<RawVariable>()
                               .DisableInitialization<IfrsVariable>()
                               .DisableInitialization<DataNodeState>());

    var committedStates = await options.TargetDataSource.Query<DataNodeState>().ToArrayAsync();
    myCommittedStates = committedStates;
    var log = await UploadDataNodeStateToWorkspaceAsync(dataSet, primaryArgs, workspace, options.TargetDataSource);
    if(log.Errors.Any()) return Activity.Finish().Merge(log); 
    var toCommitStates = (await workspace.Query<DataNodeState>().ToArrayAsync()).Except(committedStates, StatesComparer.Instance()).ToArray();
    myToCommitStates = toCommitStates;
    if (!toCommitStates.Any()) {
        ApplicationMessage.Log(Warning.VariablesAlreadyImported);   
        return Activity.Finish().Merge(log);
    }

    var allArgs = await GetAllArgsAsync(primaryArgs, options.TargetDataSource, ImportFormats.DataNodeState);
    myallArgs = allArgs;
    //var targetDataNodes = toCommitStates.Where(x => x.State == State.Active).Select(x => x.DataNode).ToHashSet();
    //mytargetDataNodes = targetDataNodes;
    var workspaceToCompute = Workspace.CreateNew();
    workspaceToCompute.Initialize(x => x.FromSource(options.TargetDataSource).DisableInitialization<RawVariable>());
    return Activity.Finish().Merge(log);
    foreach (var args in allArgs) { 
        await CommitPartitionAsync<PartitionByReportingNodeAndPeriod>(args, workspace, workspaceToCompute);
        var targetPartition = (Guid)(await options.TargetDataSource.Partition.GetKeyForInstanceAsync<PartitionByReportingNodeAndPeriod>(args));
        var defaultPartition = (Guid)(await options.TargetDataSource.Partition.GetKeyForInstanceAsync<PartitionByReportingNodeAndPeriod>(args with {Scenario = null}));
        var previousPartition = (Guid)(await options.TargetDataSource.Partition.GetKeyForInstanceAsync<PartitionByReportingNodeAndPeriod>(args with {Scenario = null, Year = args.Year-1, Month = 12}));
        await options.TargetDataSource.Partition.SetAsync<PartitionByReportingNodeAndPeriod>(null);
        if(ApplicationMessage.HasErrors()) return Activity.Finish().Merge(log);
        
        // Avoid starting the computation if no best estimate cash flow or actuals has ever been imported 
        if(!(await options.TargetDataSource.Query<RawVariable>().Where(x => x.Partition == defaultPartition).Take(1).ToArrayAsync()).Any()) continue;

        // Rawvariables and ifrsvariables corresponding to the target data nodes are added to the workspace
        var rawvariable = await options.TargetDataSource.Query<RawVariable>().Where(//x => targetDataNodes.Contains(x.DataNode) && 
                            x.Partition == targetPartition || x.Partition == defaultPartition || x.Partition == previousPartition).ToArrayAsync();
        var ifrsvariable = await options.TargetDataSource.Query<IfrsVariable>().Where(//x => targetDataNodes.Contains(x.DataNode) && 
                            x.Partition == targetPartition || x.Partition == defaultPartition || x.Partition == previousPartition).ToArrayAsync();
        
        if(rawvariable.Any()) await workspaceToCompute.UpdateAsync(rawvariable);
        if(ifrsvariable.Any()) await workspaceToCompute.UpdateAsync(ifrsvariable);

        log = log.Merge(await ComputeAsync(args, workspace, workspaceToCompute, false));
        if(log.Errors.Any()) return Activity.Finish().Merge(log);
    }

    await workspaceToCompute.UpdateAsync(toCommitStates);
    await workspaceToCompute.CommitToTargetAsync(options.TargetDataSource);
    await workspace.CommitToAsync<DataNodeState,PartitionByReportingNode>(options.TargetDataSource, 
                    (Guid)workspace.Partition.GetCurrent(nameof(PartitionByReportingNode)), snapshot: false);    
    return log;
});

In [0]:
await Import.FromString(dataNodeStateActiveScenario).WithFormat(ImportFormats.DataNodeState).WithTarget(DataSource).ExecuteAsync()

In [0]:
await DataSource.Query<DataNodeState>().ToArrayAsync()

In [0]:
myToCommitStates

In [0]:
mytargetDataNodes

In [0]:
myallArgs

In [0]:
await DataSource.Partition.SetAsync<PartitionByReportingNodeAndPeriod>(argsScenarioActive);
var dnsScenarioAct = await DataSource.Query<DataNodeState>().ToArrayAsync();

In [0]:
dnsScenarioAct

In [0]:
await DataSource.Partition.SetAsync<PartitionByReportingNodeAndPeriod>(argsBestEstimate);
var dnsBestEstimateAct = await DataSource.Query<DataNodeState>().ToArrayAsync();

In [0]:
dnsBestEstimateAct

In [0]:
var diffs = ivsScenarioParameter.Except(ivsBestEstimateInit, IfrsVariableComparer.Instance()).ToArray();

In [0]:
diffs.Count().Should().NotBe(0);
diffs.Where(x => x.EstimateType == EstimateTypes.C).Count().Should().NotBe(0);

# Re-import of the same parameters should not trigger calculations 

In [0]:
var log = await Import.FromString(parametersScenarios).WithFormat(ImportFormats.DataNodeParameter).WithTarget(DataSource).ExecuteAsync();
log

In [0]:
((log.Warnings.FirstOrDefault()?.ToString().Remove(0,40).Remove(125,2)?? "") == Get(Warning.VariablesAlreadyImported)).Should().Be(true);

In [0]:
await DataSource.Partition.SetAsync<PartitionByReportingNodeAndPeriod>(argsScenarioParameter);
var ivsScenarioParameterSameReimport = await DataSource.Query<IfrsVariable>().ToArrayAsync();

In [0]:
var diffs = ivsScenarioParameterSameReimport.Except(ivsScenarioParameter, IfrsVariableComparer.Instance());

In [0]:
diffs.Count().Should().Be(0);

# Re-import of best estimate cashflow

In [0]:
var cashflowBestEstimateReImport = @"
@@Main
ReportingNode,Year,Month,Scenario
CH,2020,12,
@@Cashflow
DataNode,AmountType,EstimateType,AocType,Novelty,AccidentYear,Values0,Values1,Values2,Values3,Values4,Values5,Values6,Values7,Values8,Values9,Values10,Values11,Values12,Values13,Values14,Values15,Values16,Values17,Values18,Values19,Values20,Values21,Values22,Values23
DT1.1,PR,BE,CL,C,,131,0,0,111,0,0,111,0,0,111,0,0,0,111,0,0,111,0,0,111,0,0,111,0
DT1.1,NIC,BE,CL,C,,-31,-31,-31,-31,-31,-31,-31,-31,-31,-31,-31,-31,-31,-31,-31,-31,-31,-31,-31,-31,-31,-31,-31,-31
DT1.1,,CU,CL,C,,-11,-11,-11,-11,-11,-11,-11,-11,-11,-11,-11,-11,-11,-5,-5,-5,-5,-5,-5,-5,-5,-5,-5,-3
DT1.1,,RA,CL,C,,-2.6,-2.6,-2.6,-2.6,-2.6,-2.6,-2.6,-2.6,-2.6,-2.6,-2.6,-2.6,-2.6,-2.6,-2.6,-2.6,-2.6,-2.6,-2.6,-2.6,-2.6,-2.6,-2.6,-2.6
DTR1.1,PR,BE,CL,C,,51,0,0,51,0,0,51,0,0,51,0,0,51,0,0,51,0,0,51,0,0,51,0,0
DTR1.1,NIC,BE,CL,C,,13,13,13,13,13,13,13,13,13,13,13,13,13,13,13,13,13,13,13,13,13,13,13,13
DTR1.1,,CU,CL,C,,-5,-5,-5,-5,-5,-5,-5,-5,-5,-5,-5,-5,-5,-5,-5,-5,-5,-5,-5,-5,-5,-5,-5,-5
DTR1.1,,RA,CL,C,,1.31,1.31,1.31,1.31,1.31,1.31,1.31,1.31,1.31,1.31,1.31,1.31,1.31,1.31,1.31,1.31,1.31,1.31,1.31,1.31,1.31,1.31,1.31,1.31";

In [0]:
await Import.FromString(cashflowBestEstimateReImport).WithFormat(ImportFormats.Cashflow).WithTarget(DataSource).ExecuteAsync()

## Best Estimate change

In [0]:
await DataSource.Partition.SetAsync<PartitionByReportingNodeAndPeriod>(argsBestEstimate);
var ivsBestEstimateReimport = await DataSource.Query<IfrsVariable>().ToArrayAsync();

In [0]:
var diffs = ivsBestEstimateReimport.Except(ivsBestEstimateInit, IfrsVariableComparer.Instance());

In [0]:
diffs.Count().Should().NotBe(0);

## Scenario should change

In [0]:
await DataSource.Partition.SetAsync<PartitionByReportingNodeAndPeriod>(argsScenarioParameter);
var ivsScenarioParameterReimportBestEstimateCashflow = await DataSource.Query<IfrsVariable>().ToArrayAsync();

In [0]:
var diffs = ivsScenarioParameterReimportBestEstimateCashflow.Except(ivsScenarioParameter, IfrsVariableComparer.Instance());

In [0]:
diffs.Count().Should().NotBe(0);

# Re-import of scenario parameters

In [0]:
var parametersScenariosReimport = @"
@@Main
ReportingNode,Year,Month,Scenario
CH,2020,12,DUP10pct
@@SingleDataNodeParameter
DataNode,PremiumAllocation
DT1.1,0.23
@@InterDataNodeParameter
DataNode,LinkedDataNode,ReinsuranceCoverage
DT1.1,DTR1.1,0.23";

In [0]:
await Import.FromString(parametersScenariosReimport).WithFormat(ImportFormats.DataNodeParameter).WithTarget(DataSource).ExecuteAsync()

In [0]:
await DataSource.Partition.SetAsync<PartitionByReportingNodeAndPeriod>(argsScenarioParameter);
var ivsScenarioParameterReimport = await DataSource.Query<IfrsVariable>().ToArrayAsync();

In [0]:
var diffs = ivsScenarioParameterReimport.Except(ivsScenarioParameterReimportBestEstimateCashflow, IfrsVariableComparer.Instance());

In [0]:
diffs.Count().Should().NotBe(0);

# Restore constants

In [0]:
Scenarios.EnableScenario = EnableScenarioBackup;